## Laden der API Keys über eine .env-Datei

In diesem Abschnitt laden wir die benötigten API-Schlüssel für OpenAI und OpenRouter aus einer `.env`-Datei. Dies ermöglicht es uns, sensible Zugangsdaten sicher zu verwalten, ohne sie direkt im Code zu hinterlegen.

Die Bibliothek `python-dotenv` liest die `.env`-Datei aus dem Projektverzeichnis und stellt die darin definierten Umgebungsvariablen zur Verfügung. Die `.env`-Datei sollte folgendes Format haben:

```
OPENAI_API_KEY=sk-...
OPEN_ROUTER_API_KEY=sk-...
```

**Wichtig:** Die `.env`-Datei sollte in der `.gitignore` aufgeführt werden, um ein versehentliches Commit von API-Schlüsseln zu verhindern.

In [ ]:
%pip install -q python-dotenv
from dotenv import load_dotenv
load_dotenv()

## Laden der erforderlichen Abhängigkeiten

Hier importieren wir alle benötigten Python-Bibliotheken für die Arbeit mit verschiedenen LLM-APIs:

- **`openai`**: Die offizielle OpenAI-Client-Bibliothek, die wir auch für kompatible APIs (LM-Studio, OpenRouter) verwenden
- **`IPython.display`**: Für die formatierte Ausgabe von Markdown-Inhalten im Notebook
- **`json`**: Zum Parsen von JSON-Antworten des Judge-Modells
- **`os`**: Für den Zugriff auf Umgebungsvariablen (API-Schlüssel)

In [ ]:
from openai import OpenAI
from IPython.display import Markdown, display
import json
import os

## Initialisierung der LLM-Clients

In diesem Abschnitt erstellen wir drei verschiedene API-Clients, die alle die OpenAI-kompatible Schnittstelle nutzen:

1. **`openai_client`**: Direkter Zugang zur OpenAI-API (GPT-4, GPT-4o, etc.)
   - Verwendet den API-Key aus der Umgebungsvariable `OPENAI_API_KEY`

2. **`lmstudio_client`**: Verbindung zu einem lokalen LM-Studio-Server
   - Nutzt eine custom `base_url` zum lokalen Server (hier: `http://100.115.26.126:1234/v1`)
   - Ermöglicht die Nutzung von lokalen Open-Source-Modellen (DeepSeek, LLaMA, etc.)

3. **`or_client`**: Zugang zu OpenRouter für alternative Modelle
   - OpenRouter bietet Zugriff auf verschiedene Modelle (Claude, Gemini, etc.) über eine einheitliche API
   - Verwendet den API-Key aus der Umgebungsvariable `OPEN_ROUTER_API_KEY`

In [ ]:

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

lmstudio_client = OpenAI(
    base_url="http://100.115.26.126:1234/v1",  # LLM Studio Server
    api_key="lm-studio"
)

or_client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPEN_ROUTER_API_KEY"),
)

## Definition der Prompts für die Dockerfile-Analyse

Hier definieren wir die verschiedenen Prompts, die wir für die LLM-Interaktion verwenden:

- **`SYSTEM_PROMPT`**: Legt die Rolle des LLM als DevSecOps-Experten fest und gibt den Kontext für alle nachfolgenden Anfragen vor

- **`CREATE_PROMPT`**: Fordert das LLM auf, ein produktionsreifes Dockerfile für eine Flask-Anwendung zu erstellen
  - Fokus auf Best Practices wie Multi-Stage-Builds und minimale Base-Images
  - Soll kommentiert und nachvollziehbar sein

- **`REVIEW_PROMPT`**: Template für die Bewertung bestehender Dockerfiles
  - Wird mit einem konkreten Dockerfile gefüllt (via `.format()`)
  - Fordert konkrete Verbesserungsvorschläge mit Begründung an

In [ ]:
SYSTEM_PROMPT = """
Du bist ein erfahrener DevSecOps-Experte mit Fokus auf sichere Containerisierung.
"""

### Bewerte und optimiere Dockerfiles nach folgenden Kriterien:
### 1) Sicherheit (least privilege, non-root, Patching, Supply Chain, CVE-Risiko)
### 2) Performance / Image-Größe (Layer, Cache, Multi-Stage, Pinning)
### 3) Best Practices für Production (Signals, Healthcheck, Entrypoint/CMD, WSGI/ASGI)
### 4) Nachvollziehbarkeit (klare Schritte, Kommentare)


CREATE_PROMPT = """
Erstelle ein sauberes, effizientes Dockerfile für eine einfache Flask‑Webanwendung mit Python 3.11.
Nutze Best Practices (mehrstufiger Build, minimale Base‑Images) und erkläre im Kommentar kurz die Schritte.
"""

REVIEW_PROMPT = (
    "Überprüfe das folgende Dockerfile auf Verbesserungsmöglichkeiten."
    "Schlage Optimierungen (Sicherheit, Größe, Performance) vor und begründe sie.\n\n{}"
)

## Helper-Funktion für LLM-Anfragen

Die Funktion `ask_openai()` ist unsere zentrale Schnittstelle zu den verschiedenen LLM-APIs:

**Parameter:**
- `client`: Der zu verwendende API-Client (openai_client, lmstudio_client oder or_client)
- `model_name`: Name des Modells (z.B. "gpt-4o", "deepseek-r1-distill-qwen-7b", "anthropic/claude-opus-4.1")
- `prompt`: Die konkrete Aufgabenstellung für das Modell

**Funktionsweise:**
- Sendet eine Chat-Completion-Anfrage mit dem System-Prompt und User-Prompt
- Verwendet `temperature=0.2` für konsistente, deterministische Antworten
- Gibt die Textantwort des Modells zurück

Diese Funktion abstrahiert die API-Kommunikation und macht es einfach, zwischen verschiedenen Modellen zu wechseln.

In [ ]:
def ask_openai(client, model_name, prompt):
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    return response.choices[0].message.content

## Dockerfile-Generierung mit verschiedenen LLM-Modellen

In den folgenden Zellen lassen wir vier verschiedene LLM-Modelle jeweils ein Dockerfile für eine Flask-Webanwendung erstellen. Dies ermöglicht uns einen direkten Vergleich der verschiedenen Ansätze und Qualitäten:

1. **GPT-4o (OpenAI)**: Das neueste und leistungsfähigste Modell von OpenAI mit exzellentem Verständnis für DevSecOps-Praktiken

2. **LLaMA 3.2 3B (lokal via LM-Studio)**: Ein kleineres Open-Source-Modell, das lokal ausgeführt wird - interessant für Datenschutz und Kosteneffizienz

3. **Claude Opus 4.1 (via OpenRouter)**: Anthropics Top-Modell, bekannt für detaillierte und durchdachte Antworten

4. **DeepSeek R1 Distill Qwen 7B (lokal via LM-Studio)**: Ein spezialisiertes Reasoning-Modell, das schrittweise denkt

Jedes Modell erhält denselben `CREATE_PROMPT` und soll ein produktionsreifes, sicheres Dockerfile mit Best Practices erstellen.

In [ ]:
gpt_dockerfile = ask_openai(openai_client, "gpt-4o", CREATE_PROMPT)
print("\n==== GPT‑4 – generiertes Dockerfile ===\n")
display(Markdown(gpt_dockerfile))

In [ ]:
local_llama_dockerfile = ask_openai(lmstudio_client, "llama-3.2-3b-instruct", CREATE_PROMPT)
print("\n==== LLaMA – generiertes Dockerfile ===\n")
display(Markdown(local_llama_dockerfile))

In [ ]:
opus = ask_openai(or_client, "anthropic/claude-opus-4.1", CREATE_PROMPT)
print("\n==== Opus – generiertes Dockerfile ===\n")
display(Markdown(opus))

In [ ]:
local_ds_dockerfile = ask_openai(lmstudio_client, "deepseek-r1-distill-qwen-7b", CREATE_PROMPT)
print("\n==== Deepseek – generiertes Dockerfile ===\n")
display(Markdown(local_ds_dockerfile))

## Review eines generierten Dockerfiles

Nachdem wir Dockerfiles erstellen lassen haben, nutzen wir nun den `REVIEW_PROMPT`, um ein bestehendes Dockerfile analysieren und verbessern zu lassen. 

In diesem Beispiel:
- Nehmen wir das von GPT-4o generierte Dockerfile (`gpt_dockerfile`)
- Übergeben es an GPT-4o selbst zur kritischen Überprüfung
- Fordern konkrete Verbesserungsvorschläge mit Begründung an

Dies demonstriert einen wichtigen DevSecOps-Workflow: Selbst automatisch generierte Dockerfiles sollten einem Review unterzogen werden, um:
- Sicherheitslücken zu identifizieren
- Performance-Optimierungen zu finden
- Best Practices durchzusetzen
- Die Image-Größe zu reduzieren

In [ ]:
review_gpt = ask_openai(openai_client, "gpt-4o", REVIEW_PROMPT.format(gpt_dockerfile))
display(Markdown(review_gpt))

In [ ]:
review_opus_ds = ask_openai(or_client, "anthropic/claude-opus-4.1", REVIEW_PROMPT.format(local_ds_dockerfile))
display(Markdown(review_opus_ds))

## LLM as Judge: Automatisierte Bewertung der Dockerfile-Qualität

Um die verschiedenen von den LLMs generierten Dockerfiles objektiv zu vergleichen, implementieren wir das **"LLM as Judge"**-Prinzip. Dabei nutzen wir ein leistungsstarkes LLM (hier: Claude Opus 4.1), um die Ausgaben der anderen Modelle zu bewerten.

### Funktionsweise:

1. **Judge-System-Prompt (`JUDGE_SYSTEM_PROMPT`)**: 
   - Definiert das LLM als unparteiischen technischen Gutachter
   - Legt 5 Bewertungskriterien fest (correctness, security, best_practices, clarity, actionability)
   - Fordert strukturierte JSON-Ausgabe mit Scores von 0-5

2. **Workflow der Bewertung**:
   - `build_judge_prompt()`: Kombiniert die ursprüngliche Aufgabe mit allen Modellantworten
   - `evaluate_models_with_openai_judge()`: Sendet die Anfrage an das Judge-Modell und parst das JSON
   - `markdown_score_table()`: Erstellt eine übersichtliche Tabelle mit Scores und Ranking
   - `compare_answers_markdown()`: High-Level-Funktion, die alles zusammenführt

3. **Bewertungskriterien**:
   - **Correctness**: Technische Korrektheit der Dockerfile-Syntax und Befehle
   - **Security**: Sicherheitsaspekte (Non-Root, Updates, Supply Chain)
   - **Best Practices**: Production-Ready-Features (Multi-Stage, Layering, Healthchecks)
   - **Clarity**: Verständlichkeit und Nachvollziehbarkeit
   - **Actionability**: Konkrete, direkt umsetzbare Vorschläge

### Vorteil:
Statt subjektiv zu entscheiden, welches Dockerfile besser ist, erhalten wir eine datenbasierte, reproduzierbare Bewertung mit detailliertem Feedback zu jedem Kriterium.

In [ ]:
# LLM as Judge

# ------------------------------------------------------------
# 1) Neutraler System-Prompt für den Judge
# ------------------------------------------------------------
JUDGE_SYSTEM_PROMPT = """
Du bist ein unparteiischer, sehr strenger technischer Gutachter für sichere Containerisierung.
Bewerte Antworten zu Dockerfile-Review/Erstellung nach diesen Kriterien (0–5, 5 = hervorragend):
- correctness: Technische Korrektheit der Aussagen
- security: Sicherheitsreife (Least-Privilege, Non-Root, Updates, Supply Chain)
- best_practices: Production-Best-Practices (Layering, Multi-Stage, CMD/ENTRYPOINT, Healthcheck, WSGI/ASGI)
- clarity: Verständlichkeit, Struktur, Nachvollziehbarkeit
- actionability: Konkrete, umsetzbare Vorschläge inkl. Code-Snippets

Gib NUR valide JSON zurück mit folgendem Schema:
{
  "per_model": {
    "<model_name>": {
      "scores": {
        "correctness": <0-5>,
        "security": <0-5>,
        "best_practices": <0-5>,
        "clarity": <0-5>,
        "actionability": <0-5>
      },
      "comment": "<kurzer Kommentar>"
    },
    ...
  },
  "ranking": ["<best>", "<next>", ...]
}
Kein weiteres Text-Drumherum, nur JSON.
"""

def ask_openai_judge(client, model_name, prompt: str):
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": JUDGE_SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0
    )
    return response.choices[0].message.content

def build_judge_prompt(question: str, answers: dict) -> str:
    """
    answers: { "gpt-4o-mini": "...", "llama-3": "...", ... }
    """
    parts = [f"Frage/Aufgabe:\n{question}\n", "Antworten der Modelle:"]
    for model_name, answer in answers.items():
        parts.append(f"\n### {model_name}\n{answer}")
    parts.append("\nBewerte alle Antworten nach den genannten Kriterien und gib NUR JSON zurück.")
    return "\n".join(parts)

def evaluate_models_with_openai_judge(question: str,
                                      answers: dict,
                                      openai_client,
                                      judge_model_name: str = "anthropic/claude-opus-4.1",
                                      judge_fn = ask_openai_judge) -> dict:
    """
    question: die gestellte Aufgabe/Frage (z. B. Dockerfile-Review)
    answers:  { model_name: model_output, ... }
    openai_client: dein bereits initialisierter OpenAI-Client
    judge_model_name: welches OpenAI-Modell den Judge spielen soll
    judge_fn: Funktions-Handle (standardmäßig ask_openai_judge)
    """
    judge_prompt = build_judge_prompt(question, answers)
    raw = judge_fn(openai_client, judge_model_name, judge_prompt)

   
    try:
        data = json.loads(raw)
    except json.JSONDecodeError:
        start = raw.find("{")
        end = raw.rfind("}")
        if start != -1 and end != -1 and end > start:
            data = json.loads(raw[start:end+1])
        else:
            raise ValueError(f"Judge antwortete nicht mit JSON. Antwort war:\n{raw}")

    # Sanity check
    if "per_model" not in data or not isinstance(data["per_model"], dict):
        raise ValueError(f"JSON hat unerwartete Struktur:\n{data}")

    return data

def markdown_score_table(eval_json: dict) -> str:
    """
    eval_json: Struktur wie vom Judge gefordert.
    Gibt eine Markdown-Tabelle mit Durchschnittsscore + Kriterien zurück.
    """
    per_model = eval_json.get("per_model", {})
    ranking   = eval_json.get("ranking", [])

    headers = ["Modell", "Gesamt", "correctness", "security", "best_practices", "clarity", "actionability", "Kommentar"]
    md = ["|" + "|".join(headers) + "|", "|" + "|".join(["---"] * len(headers)) + "|"]

    # Reihenfolge: ranking zuerst, dann restliche
    ordered_models = list(ranking) + [m for m in per_model.keys() if m not in ranking]

    for m in ordered_models:
        entry = per_model.get(m, {})
        scores = entry.get("scores", {})
        com = entry.get("comment", "")
        # Durchschnitt
        crits = ["correctness","security","best_practices","clarity","actionability"]
        vals = [scores.get(c, 0) for c in crits]
        avg = sum(vals)/len(vals) if vals else 0.0

        row = [
            f"`{m}`",
            f"{avg:.2f}",
            str(scores.get("correctness","")),
            str(scores.get("security","")),
            str(scores.get("best_practices","")),
            str(scores.get("clarity","")),
            str(scores.get("actionability","")),
            com.replace("\n"," ").strip()
        ]
        md.append("|" + "|".join(row) + "|")

    # Kurzes Fazit dazu
    if ranking:
        md.append(f"\n**Ranking:** {', '.join([f'`{r}`' for r in ranking])}")
    return "\n".join(md)


def compare_answers_markdown(question: str,
                             answers: dict,  # {model_name: text}
                             openai_client,
                             judge_model_name: str = "gpt-4o-mini",
                             judge_fn = ask_openai_judge,
                             display_markdown: bool = True) -> str:
    """
    Führt Bewertung aus und gibt eine hübsche Markdown-Tabelle zurück.
    Bei display_markdown=True wird die Tabelle direkt gerendert.
    """
    result = evaluate_models_with_openai_judge(
        question=question,
        answers=answers,
        openai_client=openai_client,
        judge_model_name=judge_model_name,
        judge_fn=judge_fn
    )
    md = markdown_score_table(result)
    if display_markdown:
        display(Markdown(md))
    return md


In [ ]:
answers = {
    "deepseek-r1-distill-qwen-7b": local_ds_dockerfile,
    "llama-3.2-3b-instruct": local_llama_dockerfile,
    "gpt-4o": gpt_dockerfile,
    "opus": opus
}

_ = compare_answers_markdown(
    question=CREATE_PROMPT,
    answers=answers,
    openai_client=or_client,
    judge_model_name="google/gemini-3-pro-preview"   # oder "gpt-4o", ""anthropic/claude-opus-4.1 falls freigeschaltet
)